# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [4]:
import torch as th

In [5]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [6]:
import syft as sy

W0708 20:20:53.804491 11188 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\Vilas\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0708 20:20:53.966788 11188 deprecation_wrapper.py:119] From C:\Users\Vilas\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [7]:
hook = sy.TorchHook(th)

In [8]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [9]:
import torch as th

In [10]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [11]:
y = x + x

In [9]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [12]:
import syft as sy

In [13]:
hook = sy.TorchHook(th)

W0708 20:21:51.931351 11188 hook.py:98] Torch was already hooked... skipping hooking process


In [14]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [15]:
bob = sy.VirtualWorker(hook, id="bob")

In [16]:
bob._objects

{}

In [17]:
x = th.tensor([1,2,3,4,5])

In [18]:
x = x.send(bob)

In [19]:
bob._objects

{20669059967: tensor([1, 2, 3, 4, 5])}

In [20]:
x.location

<VirtualWorker id:bob #objects:1>

In [21]:
x.id_at_location

20669059967

In [22]:
x.id

62670495170

In [23]:
x.owner

<VirtualWorker id:me #objects:0>

In [24]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [25]:
x

(Wrapper)>[PointerTensor | me:62670495170 -> bob:20669059967]

In [26]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [27]:
bob._objects

{}

In [28]:
bob = sy.VirtualWorker(hook, id="bob")

In [29]:
bob._objects

{}

In [30]:
x = th.tensor([1,2,3,4,5])

In [31]:
x = x.send(bob)

In [32]:
bob._objects

{98898114499: tensor([1, 2, 3, 4, 5])}

In [33]:
x.location

<VirtualWorker id:bob #objects:1>

In [34]:
x.id_at_location

98898114499

In [35]:
x.id

38173508065

In [36]:
x.owner

<VirtualWorker id:me #objects:0>

In [37]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [38]:
x

(Wrapper)>[PointerTensor | me:38173508065 -> bob:98898114499]

In [39]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [40]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [41]:
# try this project here!
alice = sy.VirtualWorker(hook, id="alice")

In [42]:
x = th.tensor([1,2,3,4,5])

In [43]:
x_ptr = x.send(bob,alice)

In [44]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [45]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [46]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

In [47]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [48]:
x

(Wrapper)>[PointerTensor | me:77446545326 -> bob:34569548400]

In [49]:
y

(Wrapper)>[PointerTensor | me:87604892030 -> bob:25147430007]

In [50]:
z = x + y
z

(Wrapper)>[PointerTensor | me:13277898755 -> bob:6360501555]

In [51]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [52]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:83135299711 -> bob:27572095561]

In [53]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [54]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [55]:
z = (x+y).sum()

In [56]:
z.backward()

(Wrapper)>[PointerTensor | me:89554749357 -> bob:50288072670]

In [57]:
x = x.get()
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [58]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Lesson: Introducing Remote Arithmetic

In [59]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [60]:
x

(Wrapper)>[PointerTensor | me:88596762433 -> bob:46885902147]

In [61]:
y

(Wrapper)>[PointerTensor | me:47777174782 -> bob:3133627756]

In [62]:
z = x + y

In [61]:
z

(Wrapper)>[PointerTensor | me:57095415431 -> bob:76472731552]

In [63]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [64]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:53315656073 -> bob:21719553922]

In [65]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [66]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [67]:
z = (x + y).sum()

In [68]:
z.backward()

(Wrapper)>[PointerTensor | me:23314346285 -> bob:44517420054]

In [69]:
x = x.get()

In [70]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [71]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [71]:
# try this project here!
input = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [72]:
weights = th.tensor([[0.],[0.]],requires_grad=True).send(bob)

In [73]:
pred = input.mm(weights)

In [74]:
loss = ((pred - target)**2).sum()

In [75]:
loss.backward()

(Wrapper)>[PointerTensor | me:10147849570 -> bob:75396250951]

In [76]:
weights.data.sub_(weights.grad * 0.1)
weights.grad *=0
print(loss.get().data)

tensor(2.)


In [77]:
for i in range(10):
    pred = input.mm(weights)
    
    loss = ((pred - target)**2).sum()
    
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *=0

    print(loss.get().data)

tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)
tensor(0.0058)


In [ ]:
bob = bob.clear_objects()

In [ ]:
bob._objects

In [ ]:
import torch as th
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
x = "asdf"

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
x

In [ ]:
bob._objects

# Lesson: Garbage Collection and Common Errors


In [ ]:
bob = bob.clear_objects()

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
del x

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
x = "asdf"

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x

In [ ]:
bob._objects

In [ ]:
x = "asdf"

In [ ]:
bob._objects

In [ ]:
del x

In [ ]:
bob._objects

In [ ]:
bob = bob.clear_objects()
bob._objects

In [ ]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [ ]:
z = x + y

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [ ]:
z = x + y

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [78]:
from torch import nn, optim

In [79]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [80]:
# A Toy Model
model = nn.Linear(2,1)

In [81]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [82]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(9.2485)
tensor(0.8306)
tensor(0.1547)
tensor(0.0756)
tensor(0.0503)
tensor(0.0352)
tensor(0.0250)
tensor(0.0179)
tensor(0.0129)
tensor(0.0093)
tensor(0.0068)
tensor(0.0050)
tensor(0.0037)
tensor(0.0027)
tensor(0.0020)
tensor(0.0015)
tensor(0.0011)
tensor(0.0009)
tensor(0.0006)
tensor(0.0005)


In [83]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [84]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [85]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [86]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [87]:
train()

tensor(0.2016, requires_grad=True)
tensor(0.4272, requires_grad=True)
tensor(0.1780, requires_grad=True)
tensor(0.2419, requires_grad=True)
tensor(0.1058, requires_grad=True)
tensor(0.1404, requires_grad=True)
tensor(0.0613, requires_grad=True)
tensor(0.0817, requires_grad=True)
tensor(0.0354, requires_grad=True)
tensor(0.0475, requires_grad=True)
tensor(0.0205, requires_grad=True)
tensor(0.0277, requires_grad=True)
tensor(0.0118, requires_grad=True)
tensor(0.0162, requires_grad=True)
tensor(0.0068, requires_grad=True)
tensor(0.0094, requires_grad=True)
tensor(0.0040, requires_grad=True)
tensor(0.0055, requires_grad=True)
tensor(0.0023, requires_grad=True)
tensor(0.0032, requires_grad=True)
tensor(0.0013, requires_grad=True)
tensor(0.0019, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0011, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0001, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [75]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [76]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [1]:
import torch as th

In [2]:
import syft as sy

W0708 22:02:35.160285 12016 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\Vilas\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0708 22:02:35.236302 12016 deprecation_wrapper.py:119] From C:\Users\Vilas\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [3]:
hook = sy.TorchHook(th)

In [4]:
bob = sy.VirtualWorker(hook, id="bob")

In [5]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [6]:
x

(Wrapper)>[PointerTensor | me:25148495371 -> bob:3660989650]

In [7]:
bob._objects

{3660989650: tensor([1, 2, 3, 4, 5])}

In [8]:
alice = sy.VirtualWorker(hook, id="alice")

In [9]:
x = x.send(alice)

In [10]:
alice._objects

{25148495371: (Wrapper)>[PointerTensor | alice:25148495371 -> bob:3660989650]}

In [11]:
x

(Wrapper)>[PointerTensor | me:43857483460 -> alice:25148495371]

In [12]:
y = x+x

In [13]:
y

(Wrapper)>[PointerTensor | me:32789229671 -> alice:6295464847]

In [14]:
bob._objects

{3660989650: tensor([1, 2, 3, 4, 5]),
 71200057346: tensor([ 2,  4,  6,  8, 10])}

In [15]:
alice._objects

{25148495371: (Wrapper)>[PointerTensor | alice:25148495371 -> bob:3660989650],
 6295464847: (Wrapper)>[PointerTensor | alice:6295464847 -> bob:71200057346]}

In [16]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:25148495371 -> bob:3660989650]

In [17]:
bob._objects

{3660989650: tensor([1, 2, 3, 4, 5]),
 71200057346: tensor([ 2,  4,  6,  8, 10])}

In [18]:
alice._objects

{6295464847: (Wrapper)>[PointerTensor | alice:6295464847 -> bob:71200057346]}

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [25]:
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{71200057346: tensor([ 2,  4,  6,  8, 10])}

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x = x.send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
y = x + x

In [ ]:
y

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
jon = sy.VirtualWorker(hook, id="jon")

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = x.get()
x

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = x.get()
x

In [ ]:
bob._objects

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
del x

In [ ]:
bob._objects

In [ ]:
alice._objects

# Lesson: Pointer Chain Operations

In [24]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [25]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [26]:
bob._objects

{490360083: tensor([1, 2, 3, 4, 5])}

In [27]:
alice._objects

{23696392077: (Wrapper)>[PointerTensor | alice:23696392077 -> bob:490360083]}

In [28]:
x.remote_get()

(Wrapper)>[PointerTensor | me:51410169970 -> alice:23696392077]

In [29]:
bob._objects

{}

In [31]:
alice._objects

{23696392077: tensor([1, 2, 3, 4, 5])}

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.move(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.remote_get()

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.move(bob)

In [ ]:
x

In [ ]:
bob._objects

In [ ]:
alice._objects